# Sai_Assignment 3

In this assignment, we will focus on healthcare. This data set is made available by MIT. It contains data about 9,026 heartbeat measurements. Each row represents a single measurement (captured on a timeline). There are a total of 80 data points (columns). This is a multiclass classification task: predict whether the measurement represents a normal heartbeat or other anomalies. 

## Description of Variables

You will use the **hearbeat_cleaned.csv** data set for this assignment. Each row represents a single measurement. Columns labeled as T1 from T80 are the time steps on the timeline (there are 80 time steps, each time step has only one measurement). 

The last column is the target variable. It shows the label (category) of the measurement as follows:<br>
0 = Normal<br>
1 = Supraventricular premature beat<br>
2 = Premature ventricular contraction<br>
3 = Fusion of ventricular and normal beat<br>
4 = Unclassifiable beat

## Goal

Use the data set **hearbeat_cleaned.csv** to predict the column called **Target**. The input variables are columns labeled as **T1 to T80**. 

## Submission:

Please save and submit this Jupyter notebook file. The correctness of the code matters for your grade. **Readability and organization of your code is also important.** You may lose points for submitting unreadable/undecipherable code. Therefore, use markdown cells to create sections, and use comments where necessary.


# Note:

The data is cleaned up. There are no uneueal length sequences. And, there is no zero padding. So, you shouldn't use any `Masking` layer (like I mentioned in the lecture). 

# Read and Prepare the Data (1 points)

In [3]:
# Common imports
import numpy as np
import pandas as pd
np.random.seed(42)

In [4]:
heartbeat = pd.read_csv("heartbeat_cleaned.csv")
heartbeat.head()

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T72,T73,T74,T75,T76,T77,T78,T79,T80,Target
0,0.987,0.892,0.461,0.1130,0.1490,0.1900,0.1650,0.1620,0.1470,0.1380,...,0.1970,0.1970,0.1960,0.2030,0.201,0.1990,0.2010,0.205,0.2080,0
1,1.000,0.918,0.621,0.1330,0.1050,0.1250,0.1170,0.0898,0.0703,0.0781,...,0.1950,0.1910,0.1520,0.1720,0.207,0.2110,0.2070,0.207,0.1720,0
2,1.000,0.751,0.143,0.1040,0.0961,0.0519,0.0442,0.0416,0.0364,0.0857,...,0.2260,0.2420,0.2440,0.2860,0.468,0.8160,0.9770,0.452,0.0519,0
3,1.000,0.740,0.235,0.0464,0.0722,0.0567,0.0103,0.0155,0.0284,0.0155,...,0.0851,0.0747,0.0515,0.0593,0.067,0.0361,0.1210,0.451,0.8690,0
4,1.000,0.833,0.309,0.0191,0.1010,0.1200,0.1040,0.0874,0.0765,0.0765,...,0.2050,0.4210,0.8030,0.9510,0.467,0.0000,0.0519,0.082,0.0628,0


In [5]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(heartbeat, test_size=0.3)
train_set.shape

(5572, 81)

In [6]:
test_set.shape

(2388, 81)

In [21]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

## Separate the target variable (we don't want to transform it)

In [8]:
train_y = train_set[['Target']]
test_y = test_set[['Target']]

train_x = train_set.drop(['Target'], axis=1)
test_x = test_set.drop(['Target'], axis=1)

# Data Transformation

In [9]:
#Target variables need to be an array with integer type
train_y = np.array(train_y)
test_y = np.array(test_y)

train_y = train_y.astype(np.int32)
test_y = test_y.astype(np.int32)


In [10]:
#Check the first 10 values of the train_y data set
train_y[0:10]


array([[1],
       [0],
       [2],
       [0],
       [2],
       [2],
       [0],
       [0],
       [4],
       [4]])

In [11]:
#Convert input variables to a 2-D array with float data type
train_x= np.array(train_x)
test_x= np.array(test_x)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [12]:
train_x

array([[1.    , 0.818 , 0.133 , ..., 0.354 , 0.354 , 0.387 ],
       [1.    , 0.849 , 0.166 , ..., 0.108 , 0.0811, 0.0695],
       [0.    , 0.0335, 0.163 , ..., 0.62  , 0.624 , 0.606 ],
       ...,
       [1.    , 0.98  , 0.573 , ..., 0.057 , 0.038 , 0.0418],
       [0.564 , 0.512 , 0.468 , ..., 0.357 , 0.348 , 0.354 ],
       [0.799 , 0.683 , 0.564 , ..., 0.265 , 0.265 , 0.247 ]],
      dtype=float32)

In [13]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))

In [14]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((5572, 80, 1), (5572, 1), (2388, 80, 1), (2388, 1))

# Find the baseline (0.5 point)

In [35]:
heartbeat[['Target']].value_counts()/len(heartbeat[['Target']])

Target
0         0.582035
4         0.198995
2         0.155402
1         0.055905
3         0.007663
dtype: float64

In [22]:
#What is your input shape?
#(meaning: how many neurons should be in the input layer?)

train_x.shape

(5572, 80, 1)

# Build a cross-sectional shallow model using Keras (with only one hidden layer) (2 points)

In [23]:
#Define the model: for multi-class
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[80, 1]),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
    
])
#final layer: there has to be 5 nodes with softmax (because we have 5 categories)


In [24]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=50,
                    validation_data=(test_x, test_y))

Epoch 1/50
175/175 [==============================] - 1s 1ms/step - loss: 0.6945 - accuracy: 0.7656 - val_loss: 0.6032 - val_accuracy: 0.8250
Epoch 2/50
175/175 [==============================] - 0s 765us/step - loss: 0.4629 - accuracy: 0.8471 - val_loss: 0.4533 - val_accuracy: 0.8413
Epoch 3/50
175/175 [==============================] - 0s 759us/step - loss: 0.3923 - accuracy: 0.8753 - val_loss: 1.0576 - val_accuracy: 0.5879
Epoch 4/50
175/175 [==============================] - 0s 736us/step - loss: 0.3649 - accuracy: 0.8855 - val_loss: 0.4029 - val_accuracy: 0.8601
Epoch 5/50
175/175 [==============================] - 0s 747us/step - loss: 0.3300 - accuracy: 0.8981 - val_loss: 0.2963 - val_accuracy: 0.9150
Epoch 6/50
175/175 [==============================] - 0s 742us/step - loss: 0.3093 - accuracy: 0.9063 - val_loss: 0.8719 - val_accuracy: 0.7513
Epoch 7/50
175/175 [==============================] - 0s 730us/step - loss: 0.3021 - accuracy: 0.9061 - val_loss: 0.4390 - val_accuracy: 0

In [27]:
# evaluate the model
scores_train = model.evaluate(train_x, train_y, verbose=0)
scores_test = model.evaluate(test_x, test_y, verbose=0)
scores_train


# In results, first is loss, second is accuracy

[0.11406494677066803, 0.9628499746322632]

In [28]:
scores_test

[0.21628998219966888, 0.9447236061096191]

In [29]:
# extract the accuracy from model.evaluate

# extract the accuracy from model.evaluate
print("%s: %.2f" % (model.metrics_names[0], scores_train[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores_train[1]*100))
print("%s: %.2f" % (model.metrics_names[0], scores_test[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores_test[1]*100))


loss: 0.11
accuracy: 96.28%
loss: 0.22
accuracy: 94.47%


# Build a cross-sectional deep model using Keras (with two or more hidden layers) (2 points)

In [30]:
#Define the model: for multi-class
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[80, 1]),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
    
])
#final layer: there has to be 5 nodes with softmax (because we have 5 categories)

In [31]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=50,
                    validation_data=(test_x, test_y))

Epoch 1/50
175/175 [==============================] - 1s 1ms/step - loss: 0.7080 - accuracy: 0.7484 - val_loss: 0.4619 - val_accuracy: 0.8291
Epoch 2/50
175/175 [==============================] - 0s 874us/step - loss: 0.4552 - accuracy: 0.8410 - val_loss: 0.6547 - val_accuracy: 0.7312
Epoch 3/50
175/175 [==============================] - 0s 891us/step - loss: 0.3817 - accuracy: 0.8798 - val_loss: 0.5792 - val_accuracy: 0.8199
Epoch 4/50
175/175 [==============================] - 0s 903us/step - loss: 0.3660 - accuracy: 0.8824 - val_loss: 0.3810 - val_accuracy: 0.8794
Epoch 5/50
175/175 [==============================] - 0s 851us/step - loss: 0.3313 - accuracy: 0.8957 - val_loss: 0.2701 - val_accuracy: 0.9255
Epoch 6/50
175/175 [==============================] - 0s 891us/step - loss: 0.2903 - accuracy: 0.9092 - val_loss: 0.8484 - val_accuracy: 0.8287
Epoch 7/50
175/175 [==============================] - 0s 885us/step - loss: 0.2932 - accuracy: 0.9085 - val_loss: 0.5405 - val_accuracy: 0

In [32]:
# evaluate the model
scores_train = model.evaluate(train_x, train_y, verbose=0)
scores_test = model.evaluate(test_x, test_y, verbose=0)
scores_train

# In results, first is loss, second is accuracy

[0.13569949567317963, 0.9574658870697021]

In [33]:
scores_test

[0.21879437565803528, 0.944304883480072]

In [34]:
# extract the accuracy from model.evaluate

# extract the accuracy from model.evaluate
print("%s: %.2f" % (model.metrics_names[0], scores_train[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores_train[1]*100))
print("%s: %.2f" % (model.metrics_names[0], scores_test[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores_test[1]*100))

loss: 0.14
accuracy: 95.75%
loss: 0.22
accuracy: 94.43%


# Build a sequential shallow LSTM Model (with only one LSTM layer) (2 points)

In [78]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(5, activation='softmax' , input_shape=[n_steps, n_inputs])
])

In [79]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [80]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=50,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/50
175/175 [==============================] - 3s 12ms/step - loss: 1.1816 - accuracy: 0.5777 - val_loss: 1.0490 - val_accuracy: 0.5984
Epoch 2/50
175/175 [==============================] - 2s 11ms/step - loss: 1.0640 - accuracy: 0.5926 - val_loss: 1.0549 - val_accuracy: 0.5942
Epoch 3/50
175/175 [==============================] - 2s 12ms/step - loss: 1.0562 - accuracy: 0.6012 - val_loss: 1.0607 - val_accuracy: 0.6051
Epoch 4/50
175/175 [==============================] - 2s 12ms/step - loss: 1.0492 - accuracy: 0.6034 - val_loss: 1.0471 - val_accuracy: 0.6160
Epoch 5/50
175/175 [==============================] - 2s 12ms/step - loss: 1.0443 - accuracy: 0.6079 - val_loss: 1.0754 - val_accuracy: 0.5637
Epoch 6/50
175/175 [==============================] - 2s 12ms/step - loss: 1.0401 - accuracy: 0.6138 - val_loss: 1.0218 - val_accuracy: 0.6118
Epoch 7/50
175/175 [==============================] - 2s 12ms/step - loss: 1.0361 - accuracy: 0.6143 - val_loss: 1.0140 - val_accuracy: 0.6185

In [81]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.7786266207695007, 0.714405357837677]

In [82]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.78
accuracy: 71.44%


# Build a sequential deep LSTM Model (with only two LSTM layers) (2 points)

In [50]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(5, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(5, return_sequences=False),
    keras.layers.Dense(5, activation='softmax')
])

In [65]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [52]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 7s 26ms/step - loss: 1.0766 - accuracy: 0.6025 - val_loss: 0.9333 - val_accuracy: 0.6327
Epoch 2/20
175/175 [==============================] - 4s 24ms/step - loss: 0.9161 - accuracy: 0.6597 - val_loss: 1.0318 - val_accuracy: 0.6637
Epoch 3/20
175/175 [==============================] - 4s 24ms/step - loss: 0.8262 - accuracy: 0.7024 - val_loss: 1.5051 - val_accuracy: 0.3945
Epoch 4/20
175/175 [==============================] - 4s 24ms/step - loss: 0.8161 - accuracy: 0.7024 - val_loss: 0.7680 - val_accuracy: 0.7320
Epoch 5/20
175/175 [==============================] - 4s 24ms/step - loss: 0.7546 - accuracy: 0.7367 - val_loss: 0.9640 - val_accuracy: 0.6495
Epoch 6/20
175/175 [==============================] - 4s 24ms/step - loss: 0.7250 - accuracy: 0.7531 - val_loss: 1.2749 - val_accuracy: 0.5034
Epoch 7/20
175/175 [==============================] - 4s 24ms/step - loss: 0.6810 - accuracy: 0.7703 - val_loss: 0.7073 - val_accuracy: 0.7374

In [53]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5301226377487183, 0.8312395215034485]

In [54]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.53
accuracy: 83.12%


# Build a sequential shallow GRU Model (with only one GRU layer) (2 points)

In [68]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(2, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(5, activation='softmax')
])

In [69]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 4s 15ms/step - loss: 1.2040 - accuracy: 0.5556 - val_loss: 1.0889 - val_accuracy: 0.5930
Epoch 2/20
175/175 [==============================] - 2s 13ms/step - loss: 1.0703 - accuracy: 0.5743 - val_loss: 1.0254 - val_accuracy: 0.5921
Epoch 3/20
175/175 [==============================] - 2s 14ms/step - loss: 1.0322 - accuracy: 0.5829 - val_loss: 1.0441 - val_accuracy: 0.5930
Epoch 4/20
175/175 [==============================] - 2s 14ms/step - loss: 1.0221 - accuracy: 0.5831 - val_loss: 1.0014 - val_accuracy: 0.6135
Epoch 5/20
175/175 [==============================] - 2s 14ms/step - loss: 1.0188 - accuracy: 0.5992 - val_loss: 1.0270 - val_accuracy: 0.5854
Epoch 6/20
175/175 [==============================] - 3s 15ms/step - loss: 1.0175 - accuracy: 0.6129 - val_loss: 0.9967 - val_accuracy: 0.6164
Epoch 7/20
175/175 [==============================] - 2s 14ms/step - loss: 1.0161 - accuracy: 0.6161 - val_loss: 0.9959 - val_accuracy: 0.6235

In [70]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.9777449369430542, 0.6281406879425049]

In [71]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.98
accuracy: 62.81%


# Build a sequential deep GRU Model (with only two GRU layers) (2 points)

In [72]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(2, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(2, return_sequences=False),
    keras.layers.Dense(5, activation='softmax')
])

In [73]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 7s 30ms/step - loss: 1.2149 - accuracy: 0.5711 - val_loss: 1.1128 - val_accuracy: 0.5930
Epoch 2/20
175/175 [==============================] - 5s 27ms/step - loss: 1.1349 - accuracy: 0.5774 - val_loss: 1.1144 - val_accuracy: 0.5930
Epoch 3/20
175/175 [==============================] - 5s 28ms/step - loss: 1.1338 - accuracy: 0.5774 - val_loss: 1.1021 - val_accuracy: 0.5930
Epoch 4/20
175/175 [==============================] - 5s 27ms/step - loss: 1.0813 - accuracy: 0.5879 - val_loss: 1.0422 - val_accuracy: 0.6156
Epoch 5/20
175/175 [==============================] - 5s 28ms/step - loss: 1.0437 - accuracy: 0.6068 - val_loss: 1.0576 - val_accuracy: 0.5930
Epoch 6/20
175/175 [==============================] - 5s 28ms/step - loss: 1.0293 - accuracy: 0.6030 - val_loss: 1.0035 - val_accuracy: 0.6076
Epoch 7/20
175/175 [==============================] - 5s 28ms/step - loss: 1.0177 - accuracy: 0.6005 - val_loss: 0.9973 - val_accuracy: 0.6131

In [74]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.8198843598365784, 0.7022613286972046]

In [75]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.82
accuracy: 70.23%


# Discussion

## List the test values of each model you built (0.5 points)

## Which model performs the best and why? (0.5 points) 
## How does it compare to baseline? (0.5 points)

